In [1]:
import time
import requests
import pandas as pd

# =========================
# CONFIG
# =========================
TOKEN_URL = "https://login.impect.com/auth/realms/production/protocol/openid-connect/token"
BASE_API_URL = "https://api.impect.com"

USERNAME = "marclambertsmedia@gmail.com"
PASSWORD = "Meneertosti@1!"  # 🔴 rotate this ASAP

OUTPUT_XLSX = "/Users/user/IMPECT/squad_kpis_iteration_1421.xlsx"

ITERATION_ID = 1421
KPI_LANGUAGE = "en"

SLEEP_SECONDS_BETWEEN_CALLS = 0.25
TIMEOUT_SECONDS = 30
# =========================


def get_access_token(username: str, password: str) -> str:
    payload = {
        "client_id": "api",
        "grant_type": "password",
        "username": username,
        "password": password,
    }
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    r = requests.post(TOKEN_URL, data=payload, headers=headers, timeout=TIMEOUT_SECONDS)
    r.raise_for_status()
    return r.json()["access_token"]


def api_get_json(url: str, token: str) -> dict:
    headers = {"Authorization": f"Bearer {token}", "Accept": "application/json"}
    r = requests.get(url, headers=headers, timeout=TIMEOUT_SECONDS)
    r.raise_for_status()
    return r.json()


def fetch_squad_kpis(iteration_id: int, token: str):
    url = f"{BASE_API_URL}/v5/customerapi/iterations/{iteration_id}/squad-kpis"
    headers = {"Authorization": f"Bearer {token}", "Accept": "application/json"}
    r = requests.get(url, headers=headers, timeout=TIMEOUT_SECONDS)
    return r.status_code, r.json()


def fetch_iteration_squads(iteration_id: int, token: str):
    url = f"{BASE_API_URL}/v5/customerapi/iterations/{iteration_id}/squads"
    headers = {"Authorization": f"Bearer {token}", "Accept": "application/json"}
    r = requests.get(url, headers=headers, timeout=TIMEOUT_SECONDS)
    return r.status_code, r.json()


# =========================
# 0) LOGIN
# =========================
access_token = get_access_token(USERNAME, PASSWORD)
print("✅ Access token retrieved")

# =========================
# 1) KPI LOOKUP
# =========================
kpi_defs = api_get_json(
    f"{BASE_API_URL}/v5/customerapi/kpis?language={KPI_LANGUAGE}",
    access_token
)

kpi_defs_df = pd.json_normalize(kpi_defs.get("data", []))

label_col = "details.label" if "details.label" in kpi_defs_df.columns else "name"

kpi_lookup = (
    kpi_defs_df
    .rename(columns={
        "id": "kpiId",
        label_col: "kpiLabel",
        "name": "kpiTechnicalName"
    })[["kpiId", "kpiLabel", "kpiTechnicalName"]]
    .drop_duplicates()
)

# =========================
# 2) FETCH SQUADS (ID → NAME)
# =========================
sc, squads_json = fetch_iteration_squads(ITERATION_ID, access_token)
squads = squads_json.get("data", squads_json)

squads_lookup_df = pd.DataFrame([
    {
        "iterationId": ITERATION_ID,
        "squadId": s.get("id"),
        "squadName": s.get("name")
    }
    for s in squads
])

print(f"✅ Squads loaded: {len(squads_lookup_df)}")

# =========================
# 3) FETCH SQUAD KPIs (LONG)
# =========================
sc, kpis_json = fetch_squad_kpis(ITERATION_ID, access_token)
data = kpis_json.get("data", kpis_json)

rows = []

for item in data:
    squad_id = item.get("squadId")
    matches = item.get("matches")

    for kv in (item.get("kpis") or []):
        rows.append({
            "iterationId": ITERATION_ID,
            "squadId": squad_id,
            "matches": matches,
            "kpiId": kv.get("kpiId"),
            "value": kv.get("value"),
        })

kpis_long = pd.DataFrame(rows)
print(f"✅ KPI rows: {len(kpis_long)}")

# =========================
# 4) MERGE KPI + SQUAD NAMES
# =========================
kpis_long["kpiId"] = pd.to_numeric(kpis_long["kpiId"], errors="coerce").astype("Int64")
kpi_lookup["kpiId"] = pd.to_numeric(kpi_lookup["kpiId"], errors="coerce").astype("Int64")

merged = (
    kpis_long
    .merge(kpi_lookup, how="left", on="kpiId")
    .merge(squads_lookup_df, how="left", on=["iterationId", "squadId"])
)

merged["value"] = pd.to_numeric(merged["value"], errors="coerce")

# =========================
# 5) CREATE WIDE TABLE
# =========================
merged["kpiColumn"] = (
    merged["kpiLabel"]
    .fillna(merged["kpiTechnicalName"])
    .fillna("kpi_" + merged["kpiId"].astype(str))
)

wide_df = (
    merged.pivot_table(
        index=["iterationId", "squadId", "squadName", "matches"],
        columns="kpiColumn",
        values="value",
        aggfunc="first"
    )
    .reset_index()
)

wide_df.columns.name = None
print(f"✅ Wide table: {wide_df.shape[0]} rows × {wide_df.shape[1]} cols")

# =========================
# 6) SAVE EXCEL
# =========================
with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl") as writer:
    merged.to_excel(writer, sheet_name="squad_kpis_long", index=False)
    wide_df.to_excel(writer, sheet_name="squad_kpis_wide", index=False)
    squads_lookup_df.to_excel(writer, sheet_name="squad_lookup", index=False)
    kpi_lookup.to_excel(writer, sheet_name="kpi_lookup", index=False)

print("\n✅ EXCEL SAVED")
print(f"📁 {OUTPUT_XLSX}")

wide_df.head()


✅ Access token retrieved
✅ Squads loaded: 16
✅ KPI rows: 21898
✅ Wide table: 16 rows × 1412 cols

✅ EXCEL SAVED
📁 /Users/user/IMPECT/squad_kpis_iteration_1421.xlsx


,iterationId,squadId,squadName,matches,,ASSISTS_AT_PHASE_ATTACKING_TRANSITION,ASSISTS_AT_PHASE_IN_POSSESSION,ASSISTS_AT_PHASE_SECOND_BALL,ASSISTS_AT_PHASE_SET_PIECE,ASSISTS_BY_ACTION_BLOCK,...,WON_GROUND_DUELS_IN_PITCH_POSITION_MIDDLE_THIRD,WON_GROUND_DUELS_IN_PITCH_POSITION_OPPONENT_BOX,WON_GROUND_DUELS_IN_PITCH_POSITION_OWN_BOX,Won Aerial Duels,Won Aerial Duels Defensive,Won Aerial Duels Offensive,Won Ground Duels,Won Ground Duels Defensive,Won Ground Duels Offensive,Yellow Card
0,1421,383,FC Slovan Liberec,19,152.684200,0.526316,0.263158,0.157895,0.526316,0.105263,...,9.263158,1.947368,1.526316,12.000000,7.631579,4.052631,25.000000,12.315789,12.368421,2.263158
1,1421,386,AC Sparta Prag,19,259.947360,0.368421,0.684211,0.105263,0.526316,NaN,...,11.526316,1.789474,1.000000,12.263158,7.842105,3.894737,25.315790,11.736842,13.052631,2.157895
2,1421,387,FC Viktoria Pilsen,19,159.473680,0.789474,0.368421,0.315789,0.263158,0.052632,...,13.368421,1.789474,1.526316,16.263159,9.578947,6.105263,31.368422,15.578947,15.105263,2.263158
3,1421,1052,FC Zlin,19,107.210526,0.473684,0.157895,0.210526,0.263158,NaN,...,9.526316,0.894737,1.842105,15.473684,7.157895,7.842105,23.578947,13.105263,9.947369,2.157895
4,1421,1053,SK Slavia Prag,19,157.421050,0.842105,0.631579,0.052632,0.526316,NaN,...,13.105263,3.105263,1.421053,19.578947,11.894737,7.052631,31.105263,14.631579,16.052631,2.052632


In [6]:
# ✅ StatsBomb-like TEAM radars (5 PNG per team) with:
# - Radar uses ACTUAL VALUES (per-match if enabled), with axis-specific min–max scales.
#   Each spoke is drawn in its own units (like your screenshot’s varying scales).
# - League average overlay uses the league MEAN in the same units.
# - Table stays: Value + Percentile (percentile is just for the table)
# - Supports MORE metrics per radar (set METRICS_PER_CATEGORY = 8/10/12 etc.)
#
# OUTPUT:
#   /Users/user/IMPECT/radars_1421/<TeamName>/<Category>.png

import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# -------------------------
# CONFIG
# -------------------------
INPUT_XLSX = "/Users/user/IMPECT/squad_kpis_iteration_1421.xlsx"
WIDE_SHEET = "squad_kpis_wide"

ITERATION_ID = 1421
OUT_DIR = "/Users/user/IMPECT/radars_1421"

USE_PER_MATCH = True

# Increase this to add more spokes per radar (if you provide more metrics)
METRICS_PER_CATEGORY = 10  # try 8, 10, 12

# tokens meaning "already normalized" (do not divide by matches)
ALREADY_NORMALIZED_TOKENS = (
    "PERCENT", "PCT", "%", "RATIO", "RATE", "ACCURACY", "AVG", "MEAN", "MEDIAN", "PER_90", "PER90"
)

# Optional header fields if present in your sheet (ignored if missing)
OPTIONAL_HEADER_FIELDS = {
    "competition": ["competition", "competitionName", "Competition", "league", "League"],
    "season": ["season", "Season"],
}

# 🔧 Provide candidate metrics per category (we’ll keep the first METRICS_PER_CATEGORY that exist)
# Add as many as you want; script will pick those found in your sheet.
CATEGORY_METRICS_CANDIDATES = {
    "Attack": [
        "SHOT_XG_AT_PHASE_IN_POSSESSION",
        "SHOT_AT_GOAL_NUMBER_AT_PHASE_IN_POSSESSION",
        "GOALS_AT_PHASE_IN_POSSESSION",
        "ASSISTS_AT_PHASE_IN_POSSESSION",
        "SUCCESSFUL_PASSES_AT_PHASE_IN_POSSESSION",
        "OFFENSIVE_TOUCHES_AT_PHASE_IN_POSSESSION",
        "KEY_PASSES_AT_PHASE_IN_POSSESSION",
        "PASSES_IN_FINAL_THIRD_NUMBER_AT_PHASE_IN_POSSESSION",
        "CROSSES_NUMBER_AT_PHASE_IN_POSSESSION",
        "DRIBBLES_SUCCESSFUL_NUMBER_AT_PHASE_IN_POSSESSION",
        "BYPASSED_OPPONENTS_NUMBER_AT_PHASE_IN_POSSESSION",
        "EXPECTED_ASSISTS_AT_PHASE_IN_POSSESSION",
    ],
    "Defence": [
        "BALL_WIN_NUMBER_AT_PHASE_OUT_OF_POSSESSION",
        "DEFENSIVE_TOUCHES_AT_PHASE_OUT_OF_POSSESSION",
        "WON_GROUND_DUELS_AT_PHASE_OUT_OF_POSSESSION",
        "WON_AERIAL_DUELS_AT_PHASE_OUT_OF_POSSESSION",
        "LOST_GROUND_DUELS_AT_PHASE_OUT_OF_POSSESSION",
        "LOST_AERIAL_DUELS_AT_PHASE_OUT_OF_POSSESSION",
        "PADJ_PRESSURES_AT_PHASE_OUT_OF_POSSESSION",
        "TACKLES_NUMBER_AT_PHASE_OUT_OF_POSSESSION",
        "INTERCEPTIONS_NUMBER_AT_PHASE_OUT_OF_POSSESSION",
        "FOULS_NUMBER_AT_PHASE_OUT_OF_POSSESSION",
        "DEFENSIVE_ACTION_OBV_AT_PHASE_OUT_OF_POSSESSION",
    ],
    "Set Piece": [
        "SHOT_XG_AT_PHASE_SET_PIECE",
        "SHOT_AT_GOAL_NUMBER_AT_PHASE_SET_PIECE",
        "GOALS_AT_PHASE_SET_PIECE",
        "ASSISTS_AT_PHASE_SET_PIECE",
        "SUCCESSFUL_PASSES_AT_PHASE_SET_PIECE",
        "OFFENSIVE_TOUCHES_AT_PHASE_SET_PIECE",
        "CROSSES_NUMBER_AT_PHASE_SET_PIECE",
        "KEY_PASSES_AT_PHASE_SET_PIECE",
        "EXPECTED_ASSISTS_AT_PHASE_SET_PIECE",
    ],
    "Transition Attack": [
        "SHOT_XG_AT_PHASE_ATTACKING_TRANSITION",
        "SHOT_AT_GOAL_NUMBER_AT_PHASE_ATTACKING_TRANSITION",
        "GOALS_AT_PHASE_ATTACKING_TRANSITION",
        "SUCCESSFUL_PASSES_AT_PHASE_ATTACKING_TRANSITION",
        "BYPASSED_OPPONENTS_NUMBER_AT_PHASE_ATTACKING_TRANSITION",
        "OFFENSIVE_TOUCHES_AT_PHASE_ATTACKING_TRANSITION",
        "DRIBBLES_SUCCESSFUL_NUMBER_AT_PHASE_ATTACKING_TRANSITION",
        "KEY_PASSES_AT_PHASE_ATTACKING_TRANSITION",
        "EXPECTED_ASSISTS_AT_PHASE_ATTACKING_TRANSITION",
    ],
    "Transition Defence": [
        "BALL_WIN_NUMBER_AT_PHASE_DEFENSIVE_TRANSITION",
        "DEFENSIVE_TOUCHES_AT_PHASE_DEFENSIVE_TRANSITION",
        "WON_GROUND_DUELS_AT_PHASE_DEFENSIVE_TRANSITION",
        "WON_AERIAL_DUELS_AT_PHASE_DEFENSIVE_TRANSITION",
        "LOST_GROUND_DUELS_AT_PHASE_DEFENSIVE_TRANSITION",
        "LOST_AERIAL_DUELS_AT_PHASE_DEFENSIVE_TRANSITION",
        "PADJ_PRESSURES_AT_PHASE_DEFENSIVE_TRANSITION",
        "TACKLES_NUMBER_AT_PHASE_DEFENSIVE_TRANSITION",
        "INTERCEPTIONS_NUMBER_AT_PHASE_DEFENSIVE_TRANSITION",
        "FOULS_NUMBER_AT_PHASE_DEFENSIVE_TRANSITION",
        "DEFENSIVE_ACTION_OBV_AT_PHASE_DEFENSIVE_TRANSITION",
    ],
}

# Metrics where "lower is better" (invert on radar and percentiles)
INVERT_METRICS = {
    "LOST_GROUND_DUELS_AT_PHASE_OUT_OF_POSSESSION",
    "LOST_AERIAL_DUELS_AT_PHASE_OUT_OF_POSSESSION",
    "LOST_GROUND_DUELS_AT_PHASE_DEFENSIVE_TRANSITION",
    "LOST_AERIAL_DUELS_AT_PHASE_DEFENSIVE_TRANSITION",
    # add: "TURNOVERS", "FOULS", "GOALS_CONCEDED", etc if present
}

# -------------------------
# STYLE (StatsBomb-ish)
# -------------------------
TEAM_COLOR = "#D64B4B"
LEAGUE_COLOR = "#7F7F7F"
HEADER_BG = "#F1F1F1"
TABLE_HEADER_BG = "#E6E6E6"
ROW_ALT_BG = "#F7F7F7"
GRID_GREY = "#D0D0D0"
RING_GREY = "#E3E3E3"
TEXT_DARK = "#333333"
TEXT_MID = "#666666"
BORDER_GREY = "#CFCFCF"

# -------------------------
# Helpers
# -------------------------
def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    return df.loc[:, ~df.columns.astype(str).str.match(r"^Unnamed:")]

def safe_filename(s: str) -> str:
    s = re.sub(r"[^A-Za-z0-9 _-]+", "", str(s)).strip()
    return re.sub(r"\s+", "_", s)[:120]

def is_already_normalized(colname: str) -> bool:
    u = str(colname).upper()
    return any(tok in u for tok in ALREADY_NORMALIZED_TOKENS)

def percentile_rank(series: pd.Series) -> pd.Series:
    s = pd.to_numeric(series, errors="coerce")
    return s.rank(pct=True, method="average")

def nice_metric_label(col: str) -> str:
    s = str(col)
    s = s.replace("_AT_PHASE_", " · ")
    s = s.replace("_BY_ACTION_", " · ")
    s = s.replace("_NUMBER", "")
    s = s.replace("_", " ")
    s = s.title()
    # light niceties
    s = s.replace("Shot Xg", "xG")
    s = s.replace("Shot At Goal", "Shots")
    s = s.replace("Successful Passes", "Succ. Passes")
    s = s.replace("Offensive Touches", "Att Touches")
    s = s.replace("Defensive Touches", "Def Touches")
    s = s.replace("Bypassed Opponents", "Opp. Bypassed")
    s = s.replace("Ball Win", "Ball Wins")
    return s

def format_value(v: float) -> str:
    if pd.isna(v):
        return ""
    v = float(v)
    av = abs(v)
    if av >= 100:
        return f"{v:.0f}"
    if av >= 10:
        return f"{v:.1f}"
    if av >= 1:
        return f"{v:.2f}"
    return f"{v:.3f}"

def compute_per_match(df: pd.DataFrame, metric_cols, matches_col="matches") -> pd.DataFrame:
    df2 = df.copy()
    m = pd.to_numeric(df2[matches_col], errors="coerce").replace(0, np.nan)
    for c in metric_cols:
        if is_already_normalized(c):
            continue
        df2[c] = pd.to_numeric(df2[c], errors="coerce") / m
    return df2

def pick_first_existing(row: pd.Series, candidates):
    for c in candidates:
        if c in row.index and pd.notna(row[c]):
            return str(row[c])
    return None

def build_percentile_frame(df_vals: pd.DataFrame, metric_cols) -> pd.DataFrame:
    out = df_vals.copy()
    for c in metric_cols:
        p = percentile_rank(out[c])
        if c in INVERT_METRICS:
            p = 1 - p
        out[c] = p
    return out

def get_category_metrics(df_cols, candidates, limit):
    found = [m for m in candidates if m in df_cols]
    return found[:limit]

# ---- Normalize each metric to 0..1 for plotting, BUT keep ring labels in actual units
def normalize_value(v, mn, mx, invert=False):
    if pd.isna(v) or pd.isna(mn) or pd.isna(mx) or mx == mn:
        return np.nan
    x = (float(v) - float(mn)) / (float(mx) - float(mn))
    if invert:
        x = 1 - x
    return np.clip(x, 0, 1)

def compute_axis_scales(df_vals: pd.DataFrame, metrics):
    """
    returns dict metric -> (min, max, mean) in actual units
    """
    scales = {}
    for m in metrics:
        s = pd.to_numeric(df_vals[m], errors="coerce")
        mn = float(s.min(skipna=True)) if s.notna().any() else np.nan
        mx = float(s.max(skipna=True)) if s.notna().any() else np.nan
        mean = float(s.mean(skipna=True)) if s.notna().any() else np.nan
        scales[m] = (mn, mx, mean)
    return scales

# -------------------------
# Drawing (StatsBomb-ish)
# -------------------------
def draw_statsbomb_header(fig, team_name, category, matches, competition=None, season=None):
    fig.patches.append(Rectangle((0, 0.92), 1, 0.08, transform=fig.transFigure,
                                 facecolor=HEADER_BG, edgecolor="none", zorder=-10))
    fig.patches.append(Rectangle((0, 0.915), 1, 0.0065, transform=fig.transFigure,
                                 facecolor=TEAM_COLOR, edgecolor="none", zorder=-9))

    fig.text(0.02, 0.963, f"{team_name}", fontsize=22, fontweight="bold",
             color=TEAM_COLOR, va="center", ha="left")

    meta_bits = []
    if competition: meta_bits.append(competition)
    if season: meta_bits.append(season)
    meta_bits.append(f"Iteration {ITERATION_ID}")
    fig.text(0.02, 0.934, "  •  ".join(meta_bits),
             fontsize=10.5, color=TEXT_MID, va="center", ha="left")

    fig.text(0.98, 0.963, f"{category} Radar",
             fontsize=12.5, fontweight="bold", color=TEXT_DARK, va="center", ha="right")
    fig.text(0.98, 0.934, f"{format_value(matches)} matches",
             fontsize=10.5, color=TEXT_MID, va="center", ha="right")

def setup_radar(ax, labels):
    n = len(labels)
    angles = np.linspace(0, 2*np.pi, n, endpoint=False)

    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)

    ax.set_xticks(angles)
    ax.set_xticklabels(labels, fontsize=9.2, color=TEXT_DARK)

    rings = [0.2, 0.4, 0.6, 0.8, 1.0]
    ax.set_yticks(rings)
    ax.set_yticklabels([""] * len(rings))  # <- no percentile labels on radar
    ax.set_ylim(0, 1)

    ax.grid(True, color=GRID_GREY, linewidth=0.9)
    ax.spines["polar"].set_color(RING_GREY)
    ax.spines["polar"].set_linewidth(1.2)
    ax.set_facecolor("white")

def plot_polygon(ax, values_0_1, color, linestyle="-", linewidth=2.2, fill_alpha=0.28, fill=True, z=3):
    vals = np.asarray(values_0_1, dtype=float)
    n = len(vals)
    ang = np.linspace(0, 2*np.pi, n, endpoint=False)
    ang_c = np.concatenate([ang, ang[:1]])
    vals_c = np.concatenate([vals, vals[:1]])

    ax.plot(ang_c, vals_c, color=color, linewidth=linewidth, linestyle=linestyle, zorder=z)
    if fill:
        ax.fill(ang_c, vals_c, color=color, alpha=fill_alpha, zorder=z-1)

def draw_table_panel(ax, table_df: pd.DataFrame):
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis("off")

    ax.add_patch(Rectangle((0.0, 0.0), 1.0, 1.0, facecolor="white", edgecolor=BORDER_GREY, linewidth=1.0))

    header_h = 0.085
    ax.add_patch(Rectangle((0.0, 1-header_h), 1.0, header_h, facecolor=TABLE_HEADER_BG, edgecolor=BORDER_GREY, linewidth=1.0))
    ax.text(0.03, 1-header_h/2, "Metric", va="center", ha="left", fontsize=10, fontweight="bold", color=TEXT_DARK)
    ax.text(0.72, 1-header_h/2, "Value", va="center", ha="right", fontsize=10, fontweight="bold", color=TEXT_DARK)
    ax.text(0.97, 1-header_h/2, "Pct", va="center", ha="right", fontsize=10, fontweight="bold", color=TEXT_DARK)

    n = len(table_df)
    if n == 0:
        return

    top = 1 - header_h
    row_h = (top - 0.05) / n
    y = top

    for i in range(n):
        y0 = y - row_h
        bg = ROW_ALT_BG if i % 2 == 0 else "white"
        ax.add_patch(Rectangle((0.0, y0), 1.0, row_h, facecolor=bg, edgecolor=BORDER_GREY, linewidth=0.5))

        metric = str(table_df.iloc[i]["Metric"])
        value = str(table_df.iloc[i]["Value"])
        pct = table_df.iloc[i]["Percentile"]

        ax.text(0.03, y0 + row_h/2, metric, va="center", ha="left", fontsize=9.4, color=TEXT_DARK)
        ax.text(0.72, y0 + row_h/2, value, va="center", ha="right", fontsize=9.4, color=TEXT_DARK)

        pct_text = "" if pct == "" or pd.isna(pct) else str(int(pct))
        pct_color = TEXT_DARK
        if pct_text != "":
            p = int(pct_text)
            if p >= 80:
                pct_color = TEAM_COLOR
            elif p >= 60:
                pct_color = "#E28B2D"
            elif p <= 20:
                pct_color = "#2B6CB0"
            else:
                pct_color = TEXT_DARK

        ax.text(0.97, y0 + row_h/2, pct_text, va="center", ha="right",
                fontsize=9.4, color=pct_color, fontweight="bold")
        y = y0

    ax.text(0.03, 0.015, "Radar: solid = team, dashed = league avg", fontsize=8.2, color=TEXT_MID, ha="left", va="bottom")

# -------------------------
# Load data
# -------------------------
df = pd.read_excel(INPUT_XLSX, sheet_name=WIDE_SHEET)
df = clean_columns(df)

required = ["squadName", "matches"]
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns in '{WIDE_SHEET}': {missing}")

df["matches"] = pd.to_numeric(df["matches"], errors="coerce")

# Select metrics per category (more spokes)
CATEGORY_METRICS = {}
all_metrics = set()
for cat, candidates in CATEGORY_METRICS_CANDIDATES.items():
    mets = get_category_metrics(df.columns, candidates, METRICS_PER_CATEGORY)
    CATEGORY_METRICS[cat] = mets
    all_metrics |= set(mets)

all_metrics = sorted(all_metrics)
if not all_metrics:
    raise ValueError("None of the configured metrics exist in your sheet. Update CATEGORY_METRICS_CANDIDATES.")

# Values (optionally per match)
df_vals = df.copy()
if USE_PER_MATCH:
    df_vals = compute_per_match(df_vals, all_metrics, matches_col="matches")

# Percentiles for table only
df_pct = build_percentile_frame(df_vals, all_metrics)

# -------------------------
# Render 5 PNGs per team
# -------------------------
os.makedirs(OUT_DIR, exist_ok=True)

for idx, row in df.iterrows():
    team_name = str(row["squadName"])
    team_dir = os.path.join(OUT_DIR, safe_filename(team_name))
    os.makedirs(team_dir, exist_ok=True)

    matches = row.get("matches", np.nan)
    competition = pick_first_existing(row, OPTIONAL_HEADER_FIELDS["competition"])
    season = pick_first_existing(row, OPTIONAL_HEADER_FIELDS["season"])

    for category, metrics in CATEGORY_METRICS.items():
        if len(metrics) < 3:
            continue

        # Axis scales in actual units (min/max/mean across teams)
        scales = compute_axis_scales(df_vals, metrics)

        # Actual team values in units
        team_vals = df_vals.loc[idx, metrics].astype(float).values

        # League mean in units
        league_means = np.array([scales[m][2] for m in metrics], dtype=float)

        # Convert both to 0..1 for plotting using per-axis min/max
        team_poly = np.array([
            normalize_value(v, scales[m][0], scales[m][1], invert=(m in INVERT_METRICS))
            for v, m in zip(team_vals, metrics)
        ], dtype=float)

        league_poly = np.array([
            normalize_value(scales[m][2], scales[m][0], scales[m][1], invert=(m in INVERT_METRICS))
            for m in metrics
        ], dtype=float)

        # Table: Value + Percentile
        pcts = df_pct.loc[idx, metrics].values
        table_df = pd.DataFrame({
            "Metric": [nice_metric_label(m) for m in metrics],
            "Value": [format_value(v) for v in team_vals],
            "Percentile": [int(round(float(p)*100)) if pd.notna(p) else "" for p in pcts],
        })

        # ---- FIGURE
        fig = plt.figure(figsize=(13, 7.2), facecolor="white")
        draw_statsbomb_header(fig, team_name, category, matches, competition=competition, season=season)

        ax_radar = fig.add_axes([0.04, 0.10, 0.54, 0.78], polar=True)
        labels = [nice_metric_label(m) for m in metrics]
        setup_radar(ax_radar, labels)

        # dashed league mean
        plot_polygon(ax_radar, league_poly, color=LEAGUE_COLOR, linestyle="--", linewidth=2.0, fill=False, z=2)
        # solid team
        plot_polygon(ax_radar, team_poly, color=TEAM_COLOR, linestyle="-", linewidth=2.6, fill=True, fill_alpha=0.30, z=3)

        # Legend line
        fig.text(0.06, 0.085, "Team", color=TEAM_COLOR, fontsize=10, fontweight="bold")
        fig.text(0.115, 0.085, "•", color=TEXT_MID, fontsize=10)
        fig.text(0.125, 0.085, "League Avg", color=LEAGUE_COLOR, fontsize=10, fontweight="bold")

        # Right table
        ax_table = fig.add_axes([0.62, 0.10, 0.35, 0.78])
        draw_table_panel(ax_table, table_df)

        # Watermark-ish
        fig.text(0.04, 0.02, "hudlstatsbomb", fontsize=16, fontweight="bold", color="#F05A28", alpha=0.35)

        # Save
        out_path = os.path.join(team_dir, f"{safe_filename(category)}.png")
        fig.savefig(out_path, dpi=240, bbox_inches="tight")
        plt.close(fig)

print(f"✅ Done. Saved ACTUAL-VALUE (axis min-max) radars to: {OUT_DIR}")
print(f"Each team folder contains up to 5 PNGs. Spokes per radar ≈ {METRICS_PER_CATEGORY} (where available).")


✅ Done. Saved ACTUAL-VALUE (axis min-max) radars to: /Users/user/IMPECT/radars_1421
Each team folder contains up to 5 PNGs. Spokes per radar ≈ 10 (where available).


In [15]:
# ✅ PERFECTED TEAM RADARS (Iteration 1421) — tuned to YOUR Excel
# - Reads /mnt/data or local path
# - 5 radars per team (Attack/Defence/Set Piece/Transition Attack/Transition Defence)
# - Removes duplicates by preferring *_NUMBER_* variants
# - Drops phase text in metric names
# - League average = dashed + light fill
# - Team = solid + fill
# - Table shows Value + COLORED Percentile
# - Ring labels show ACTUAL values (scaled per metric); to avoid clutter we label 4 rings

import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# -------------------------
# PATHS
# -------------------------
# Use your local path when running locally:
# INPUT_XLSX = "/Users/user/IMPECT/squad_kpis_iteration_1421.xlsx"
# Or use the uploaded file path in this environment:
INPUT_XLSX = "/Users/user/IMPECT/squad_kpis_iteration_1421.xlsx"
WIDE_SHEET = "squad_kpis_wide"

ITERATION_ID = 1421
OUT_DIR = "/Users/user/IMPECT/radars_1421"   # change to "/Users/user/IMPECT/radars_1421" locally

# -------------------------
# RADAR SETTINGS
# -------------------------
USE_PER_MATCH = True
SCALE_QUANTILES = (0.05, 0.95)   # robust min/max per metric; set None for strict min/max
METRICS_PER_CATEGORY = 10        # tuned: values on each spoke → keep modest

# Draw rings at these radii, label fewer rings to reduce clutter
RINGS_DRAW = [0.2, 0.4, 0.6, 0.8, 1.0]
RINGS_LABEL = [0.25, 0.5, 0.75, 1.0]   # fewer labels → much cleaner

# label placement
RING_LABEL_FONTSIZE = 7.4
RING_LABEL_RADIUS_OFFSET = 0.01

METRIC_LABEL_RADIUS = 1.12
METRIC_LABEL_FONTSIZE = 10.5
METRIC_LABEL_WRAP = 22
METRIC_LABEL_LINE_SPACING = 1.0

# If you still get clutter, set these:
# METRICS_PER_CATEGORY = 8
# RINGS_LABEL = [0.5, 1.0]

# -------------------------
# TOKEN / METADATA
# -------------------------
ALREADY_NORMALIZED_TOKENS = (
    "PERCENT", "PCT", "%", "RATIO", "RATE", "ACCURACY", "AVG", "MEAN", "MEDIAN", "PER_90", "PER90"
)

# lower is better => invert (so outward = better)
INVERT_METRICS = {
    "LOST_GROUND_DUELS_AT_PHASE_OUT_OF_POSSESSION",
    "LOST_AERIAL_DUELS_AT_PHASE_OUT_OF_POSSESSION",
    "LOST_GROUND_DUELS_AT_PHASE_DEFENSIVE_TRANSITION",
    "LOST_AERIAL_DUELS_AT_PHASE_DEFENSIVE_TRANSITION",
    "FOULS_NUMBER_AT_PHASE_OUT_OF_POSSESSION",
    "FOULS_NUMBER_AT_PHASE_DEFENSIVE_TRANSITION",
}

PHASE_TOKEN = {
    "Attack": "AT_PHASE_IN_POSSESSION",
    "Defence": "AT_PHASE_OUT_OF_POSSESSION",
    "Set Piece": "AT_PHASE_SET_PIECE",
    "Transition Attack": "AT_PHASE_ATTACKING_TRANSITION",
    "Transition Defence": "AT_PHASE_DEFENSIVE_TRANSITION",
}

NON_METRIC_COLS = {"iterationId", "season", "squadId", "squadName", "matches"}

# -------------------------
# STYLE (StatsBomb-ish)
# -------------------------
TEAM_COLOR = "#D64B4B"
LEAGUE_COLOR = "#7F7F7F"

HEADER_BG = "#F1F1F1"
TABLE_HEADER_BG = "#E6E6E6"
ROW_ALT_BG = "#F7F7F7"
GRID_GREY = "#D0D0D0"
RING_GREY = "#E3E3E3"
TEXT_DARK = "#333333"
TEXT_MID = "#666666"
BORDER_GREY = "#CFCFCF"

PCT_BLUE = "#2B6CB0"
PCT_ORANGE = "#E28B2D"
PCT_RED = TEAM_COLOR

# -------------------------
# CATEGORY “PERFECT” METRIC ORDER (based on your actual columns)
# We explicitly prefer *_NUMBER_* to prevent duplicates.
# If a listed metric is missing, we skip it and fill with good leftovers.
# -------------------------
CATEGORY_PREFERRED = {
    "Attack": [
        "SHOT_XG_AT_PHASE_IN_POSSESSION",
        "GOALS_AT_PHASE_IN_POSSESSION",
        "SHOT_AT_GOAL_NUMBER_AT_PHASE_IN_POSSESSION",
        "SHOT_AT_GOAL_OFF_TARGET_NUMBER_AT_PHASE_IN_POSSESSION",
        "ASSISTS_AT_PHASE_IN_POSSESSION",
        "BYPASSED_OPPONENTS_NUMBER_AT_PHASE_IN_POSSESSION",
        "BYPASSED_OPPONENTS_RECEIVING_NUMBER_AT_PHASE_IN_POSSESSION",
        "SUCCESSFUL_PASSES_AT_PHASE_IN_POSSESSION",
        "UNSUCCESSFUL_PASSES_AT_PHASE_IN_POSSESSION",
        "NEUTRAL_PASSES_AT_PHASE_IN_POSSESSION",
        "OFFENSIVE_TOUCHES_AT_PHASE_IN_POSSESSION",
        "REVERSE_PLAY_NUMBER_AT_PHASE_IN_POSSESSION",
        "BYPASSED_DEFENDERS_AT_PHASE_IN_POSSESSION",
        "BYPASSED_DEFENDERS_RECEIVING_AT_PHASE_IN_POSSESSION",
    ],
    "Defence": [
        "BALL_WIN_NUMBER_AT_PHASE_OUT_OF_POSSESSION",
        "BALL_WIN_NUMBER_BY_ACTION_INTERCEPTION",
        "DEFENSIVE_TOUCHES_AT_PHASE_OUT_OF_POSSESSION",
        "WON_GROUND_DUELS_AT_PHASE_OUT_OF_POSSESSION",
        "WON_AERIAL_DUELS_AT_PHASE_OUT_OF_POSSESSION",
        "LOST_GROUND_DUELS_AT_PHASE_OUT_OF_POSSESSION",
        "LOST_AERIAL_DUELS_AT_PHASE_OUT_OF_POSSESSION",
        "Number of presses",
        "Number of presses between the lines",
        "Number of presses during opponent build-up",
        "Number of presses in counter press",
    ],
    "Set Piece": [
        "SHOT_XG_AT_PHASE_SET_PIECE",
        "GOALS_AT_PHASE_SET_PIECE",
        "SHOT_AT_GOAL_NUMBER_AT_PHASE_SET_PIECE",
        "SHOT_AT_GOAL_OFF_TARGET_NUMBER_AT_PHASE_SET_PIECE",
        "ASSISTS_AT_PHASE_SET_PIECE",
        "SUCCESSFUL_PASSES_AT_PHASE_SET_PIECE",
        "UNSUCCESSFUL_PASSES_AT_PHASE_SET_PIECE",
        "NEUTRAL_PASSES_AT_PHASE_SET_PIECE",
        "BYPASSED_OPPONENTS_NUMBER_AT_PHASE_SET_PIECE",
        "BYPASSED_OPPONENTS_RECEIVING_NUMBER_AT_PHASE_SET_PIECE",
        "BALL_WIN_NUMBER_AT_PHASE_SET_PIECE",
        "WON_AERIAL_DUELS_AT_PHASE_SET_PIECE",
        "DEFENSIVE_TOUCHES_AT_PHASE_SET_PIECE",
        "OFFENSIVE_TOUCHES_AT_PHASE_SET_PIECE",
    ],
    "Transition Attack": [
        "SHOT_XG_AT_PHASE_ATTACKING_TRANSITION",
        "GOALS_AT_PHASE_ATTACKING_TRANSITION",
        "SHOT_AT_GOAL_NUMBER_AT_PHASE_ATTACKING_TRANSITION",
        "SHOT_AT_GOAL_OFF_TARGET_NUMBER_AT_PHASE_ATTACKING_TRANSITION",
        "ASSISTS_AT_PHASE_ATTACKING_TRANSITION",
        "BYPASSED_OPPONENTS_NUMBER_AT_PHASE_ATTACKING_TRANSITION",
        "BYPASSED_OPPONENTS_RECEIVING_NUMBER_AT_PHASE_ATTACKING_TRANSITION",
        "SUCCESSFUL_PASSES_AT_PHASE_ATTACKING_TRANSITION",
        "UNSUCCESSFUL_PASSES_AT_PHASE_ATTACKING_TRANSITION",
        "NEUTRAL_PASSES_AT_PHASE_ATTACKING_TRANSITION",
        "OFFENSIVE_TOUCHES_AT_PHASE_ATTACKING_TRANSITION",
        "REVERSE_PLAY_NUMBER_AT_PHASE_ATTACKING_TRANSITION",
    ],
    "Transition Defence": [
        "BALL_WIN_NUMBER_AT_PHASE_DEFENSIVE_TRANSITION",
        "DEFENSIVE_TOUCHES_AT_PHASE_DEFENSIVE_TRANSITION",
        "WON_GROUND_DUELS_AT_PHASE_DEFENSIVE_TRANSITION",
        "WON_AERIAL_DUELS_AT_PHASE_DEFENSIVE_TRANSITION",
        "LOST_GROUND_DUELS_AT_PHASE_DEFENSIVE_TRANSITION",
        "LOST_AERIAL_DUELS_AT_PHASE_DEFENSIVE_TRANSITION",
        "BALL_WIN_REMOVED_OPPONENTS_AT_PHASE_DEFENSIVE_TRANSITION",
        "BALL_WIN_ADDED_TEAMMATES_AT_PHASE_DEFENSIVE_TRANSITION",
    ],
}

# -------------------------
# Helpers
# -------------------------
def safe_filename(s: str) -> str:
    s = re.sub(r"[^A-Za-z0-9 _-]+", "", str(s)).strip()
    return re.sub(r"\s+", "_", s)[:120]

def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    return df.loc[:, ~df.columns.astype(str).str.match(r"^Unnamed:")]

def is_already_normalized(colname: str) -> bool:
    u = str(colname).upper()
    return any(tok in u for tok in ALREADY_NORMALIZED_TOKENS)

def format_value(v: float) -> str:
    if pd.isna(v):
        return ""
    v = float(v)
    av = abs(v)
    if av >= 100:
        return f"{v:.0f}"
    if av >= 10:
        return f"{v:.1f}"
    if av >= 1:
        return f"{v:.2f}"
    return f"{v:.3f}"

def pct_to_color(p):
    if p == "" or pd.isna(p):
        return TEXT_DARK
    p = int(p)
    if p >= 80:
        return PCT_RED
    if p >= 60:
        return PCT_ORANGE
    if p <= 20:
        return PCT_BLUE
    return TEXT_DARK

def _wrap_label(text: str, width: int) -> str:
    words = str(text).split()
    lines, cur = [], ""
    for w in words:
        if len(cur) + len(w) + (1 if cur else 0) <= width:
            cur = (cur + " " + w).strip()
        else:
            if cur:
                lines.append(cur)
            cur = w
    if cur:
        lines.append(cur)
    return "\n".join(lines)

def nice_metric_label(col: str) -> str:
    """
    Drops phase text like "In Possession" / "Out of Possession" etc.
    Keeps names clean.
    """
    s = str(col)

    # remove phase tokens completely
    for tok in PHASE_TOKEN.values():
        s = s.replace(tok, "")

    s = s.replace("_AT_PHASE_", " ")
    s = s.replace("_BY_ACTION_", " ")
    s = s.replace("_NUMBER", "")
    s = s.replace("_", " ")
    s = re.sub(r"\s+", " ", s).strip()

    # common short forms
    s = re.sub(r"\bSHOT XG\b", "xG", s, flags=re.IGNORECASE)
    s = re.sub(r"\bEXPECTED ASSISTS\b", "xA", s, flags=re.IGNORECASE)
    s = re.sub(r"\bSHOT AT GOAL OFF TARGET\b", "Shots Off Target", s, flags=re.IGNORECASE)
    s = re.sub(r"\bSHOT AT GOAL\b", "Shots", s, flags=re.IGNORECASE)
    s = re.sub(r"\bSUCCESSFUL PASSES\b", "Succ. Passes", s, flags=re.IGNORECASE)
    s = re.sub(r"\bUNSUCCESSFUL PASSES\b", "Unsuccessful Passes", s, flags=re.IGNORECASE)
    s = re.sub(r"\bBYPASSED OPPONENTS RECEIVING\b", "Opp. Bypassed Receiving", s, flags=re.IGNORECASE)
    s = re.sub(r"\bBYPASSED OPPONENTS\b", "Opp. Bypassed", s, flags=re.IGNORECASE)
    s = re.sub(r"\bPADJ PRESSURES\b", "pAdj Pressures", s, flags=re.IGNORECASE)
    s = re.sub(r"\bBALL WIN\b", "Ball Wins", s, flags=re.IGNORECASE)

    # Title-case (keep xG/xA/pAdj)
    s = s.title()
    s = s.replace("Xg", "xG").replace("Xa", "xA").replace("Padj", "pAdj")
    return s

def compute_per_match(df: pd.DataFrame, metric_cols, matches_col="matches") -> pd.DataFrame:
    df2 = df.copy()
    m = pd.to_numeric(df2[matches_col], errors="coerce").replace(0, np.nan)
    for c in metric_cols:
        if is_already_normalized(c):
            continue
        df2[c] = pd.to_numeric(df2[c], errors="coerce") / m
    return df2

def percentile_rank(series: pd.Series) -> pd.Series:
    s = pd.to_numeric(series, errors="coerce")
    return s.rank(pct=True, method="average")

def axis_bounds(series: pd.Series):
    s = pd.to_numeric(series, errors="coerce").dropna()
    if s.empty:
        return (np.nan, np.nan, np.nan)
    mean = float(s.mean())
    if SCALE_QUANTILES is None:
        mn = float(s.min())
        mx = float(s.max())
    else:
        qlo, qhi = SCALE_QUANTILES
        mn = float(s.quantile(qlo))
        mx = float(s.quantile(qhi))
        if mx == mn:
            mn = float(s.min())
            mx = float(s.max())
    return (mn, mx, mean)

def normalize(v, mn, mx, invert=False):
    if pd.isna(v) or pd.isna(mn) or pd.isna(mx) or mx == mn:
        return np.nan
    x = (float(v) - float(mn)) / (float(mx) - float(mn))
    x = np.clip(x, 0, 1)
    if invert:
        x = 1 - x
    return x

def ring_value_from_radius(mn, mx, r, invert=False):
    if pd.isna(mn) or pd.isna(mx) or mx == mn:
        return np.nan
    if invert:
        return mx - r * (mx - mn)
    return mn + r * (mx - mn)

def build_scales(df_vals: pd.DataFrame, metrics):
    return {m: axis_bounds(df_vals[m]) for m in metrics}

def pick_metrics_for_category(all_cols, category, limit):
    """
    Perfect selection:
    - Use preferred list first (already de-duplicated by choosing *_NUMBER_* versions)
    - Fill remaining with best leftovers from that phase (if applicable)
    - Dedupe by display label (nice_metric_label)
    """
    preferred = CATEGORY_PREFERRED.get(category, [])
    picked = []
    used_labels = set()

    # take preferred in order
    for c in preferred:
        if c in all_cols:
            lab = nice_metric_label(c)
            if lab not in used_labels:
                picked.append(c)
                used_labels.add(lab)
        if len(picked) >= limit:
            return picked

    # fallback fill from phase token columns
    token = PHASE_TOKEN.get(category)
    if token:
        leftovers = [c for c in all_cols if isinstance(c, str) and token in c and c not in picked]
        # prefer *_NUMBER_* if exists
        leftovers = sorted(leftovers, key=lambda x: (0 if "_NUMBER_" in x else 1, len(x)))
        for c in leftovers:
            lab = nice_metric_label(c)
            if lab in used_labels:
                continue
            picked.append(c)
            used_labels.add(lab)
            if len(picked) >= limit:
                break

    return picked

# -------------------------
# Drawing
# -------------------------
def draw_header(fig, team_name, category, matches):
    fig.patches.append(Rectangle((0, 0.92), 1, 0.08, transform=fig.transFigure,
                                 facecolor=HEADER_BG, edgecolor="none", zorder=-10))
    fig.patches.append(Rectangle((0, 0.915), 1, 0.0065, transform=fig.transFigure,
                                 facecolor=TEAM_COLOR, edgecolor="none", zorder=-9))

    fig.text(0.03, 0.963, f"{team_name}", fontsize=26, fontweight="bold",
             color=TEAM_COLOR, va="center", ha="left")
    fig.text(0.03, 0.934, f"Iteration {ITERATION_ID}", fontsize=12,
             color=TEXT_MID, va="center", ha="left")

    fig.text(0.97, 0.963, f"{category} Radar", fontsize=16, fontweight="bold",
             color=TEXT_DARK, va="center", ha="right")
    fig.text(0.97, 0.934, f"{format_value(matches)} matches", fontsize=12,
             color=TEXT_MID, va="center", ha="right")

def setup_radar(ax, n_spokes):
    angles = np.linspace(0, 2*np.pi, n_spokes, endpoint=False)
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)

    ax.set_xticks(angles)
    ax.set_xticklabels([""] * n_spokes)

    ax.set_yticks(RINGS_DRAW)
    ax.set_yticklabels([""] * len(RINGS_DRAW))
    ax.set_ylim(0, 1)

    ax.grid(True, color=GRID_GREY, linewidth=1.0)
    ax.spines["polar"].set_color(RING_GREY)
    ax.spines["polar"].set_linewidth(1.5)
    ax.set_facecolor("white")

def plot_polygon(ax, values_0_1, color, linestyle="-", linewidth=3.0, fill=True, fill_alpha=0.22, z=3):
    vals = np.asarray(values_0_1, dtype=float)
    n = len(vals)
    ang = np.linspace(0, 2*np.pi, n, endpoint=False)
    ang_c = np.concatenate([ang, ang[:1]])
    vals_c = np.concatenate([vals, vals[:1]])

    ax.plot(ang_c, vals_c, color=color, linewidth=linewidth, linestyle=linestyle, zorder=z)
    if fill:
        ax.fill(ang_c, vals_c, color=color, alpha=fill_alpha, zorder=z-1)

def draw_metric_titles(ax, metrics):
    n = len(metrics)
    angles = np.linspace(0, 2*np.pi, n, endpoint=False)
    for ang, m in zip(angles, metrics):
        label = _wrap_label(nice_metric_label(m), METRIC_LABEL_WRAP)
        x = np.cos(ang)
        if abs(x) < 0.15:
            ha = "center"
        else:
            ha = "left" if x > 0 else "right"

        ax.text(
            ang, METRIC_LABEL_RADIUS, label,
            fontsize=METRIC_LABEL_FONTSIZE, color=TEXT_DARK,
            ha=ha, va="center", linespacing=METRIC_LABEL_LINE_SPACING
        )

def draw_ring_value_labels_all(ax, metrics, scales):
    n = len(metrics)
    angles = np.linspace(0, 2*np.pi, n, endpoint=False)

    for ang, m in zip(angles, metrics):
        mn, mx, _mean = scales[m]
        if pd.isna(mn) or pd.isna(mx) or mx == mn:
            continue

        inv = (m in INVERT_METRICS)
        ha = "left" if np.cos(ang) >= 0 else "right"
        ang_shift = 0.015 if ha == "left" else -0.015

        for r in RINGS_LABEL:
            val = ring_value_from_radius(mn, mx, r, invert=inv)
            if pd.isna(val):
                continue
            ax.text(
                ang + ang_shift, r + RING_LABEL_RADIUS_OFFSET,
                format_value(val),
                fontsize=RING_LABEL_FONTSIZE, color=TEXT_MID,
                ha=ha, va="center"
            )

def draw_table(ax, table_df: pd.DataFrame):
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis("off")

    ax.add_patch(Rectangle((0, 0), 1, 1, facecolor="white", edgecolor=BORDER_GREY, linewidth=1.0))

    header_h = 0.09
    ax.add_patch(Rectangle((0, 1-header_h), 1, header_h, facecolor=TABLE_HEADER_BG, edgecolor=BORDER_GREY, linewidth=1.0))
    ax.text(0.04, 1-header_h/2, "Metric", va="center", ha="left", fontsize=12, fontweight="bold", color=TEXT_DARK)
    ax.text(0.74, 1-header_h/2, "Value", va="center", ha="right", fontsize=12, fontweight="bold", color=TEXT_DARK)
    ax.text(0.95, 1-header_h/2, "Pct", va="center", ha="right", fontsize=12, fontweight="bold", color=TEXT_DARK)

    n = len(table_df)
    top = 1 - header_h
    row_h = (top - 0.06) / max(n, 1)
    y = top

    for i in range(n):
        y0 = y - row_h
        bg = ROW_ALT_BG if i % 2 == 0 else "white"
        ax.add_patch(Rectangle((0, y0), 1, row_h, facecolor=bg, edgecolor=BORDER_GREY, linewidth=0.5))

        metric = str(table_df.iloc[i]["Metric"])
        value = str(table_df.iloc[i]["Value"])
        pct = table_df.iloc[i]["Percentile"]

        ax.text(0.04, y0 + row_h/2, metric, va="center", ha="left", fontsize=11, color=TEXT_DARK)
        ax.text(0.74, y0 + row_h/2, value, va="center", ha="right", fontsize=11, color=TEXT_DARK)

        pct_text = "" if pct == "" or pd.isna(pct) else str(int(pct))
        ax.text(0.95, y0 + row_h/2, pct_text, va="center", ha="right",
                fontsize=11, color=pct_to_color(pct_text), fontweight="bold")

        y = y0

    ax.text(0.04, 0.02, "Radar: solid = team, dashed+filled = league avg",
            fontsize=10, color=TEXT_MID, ha="left", va="bottom")

# -------------------------
# LOAD DATA
# -------------------------
wide = pd.read_excel(INPUT_XLSX, sheet_name=WIDE_SHEET)
wide = clean_columns(wide)

for col in ["squadName", "matches"]:
    if col not in wide.columns:
        raise ValueError(f"Missing required column '{col}' in sheet '{WIDE_SHEET}'")

wide["matches"] = pd.to_numeric(wide["matches"], errors="coerce")

all_cols = list(wide.columns)

# Pick metrics per category using the “perfect” lists + fallback fill
CATEGORY_METRICS = {cat: pick_metrics_for_category(all_cols, cat, METRICS_PER_CATEGORY) for cat in PHASE_TOKEN.keys()}

print("Final metric counts per category (after dedupe):")
for cat, mets in CATEGORY_METRICS.items():
    print(f"  - {cat}: {len(mets)} -> {[nice_metric_label(m) for m in mets]}")

# build union for normalization
all_metrics = sorted(set(m for mets in CATEGORY_METRICS.values() for m in mets))

# values frame (optionally per match)
vals = wide.copy()
if USE_PER_MATCH:
    vals = compute_per_match(vals, all_metrics, matches_col="matches")

# percentiles for table only
pct_df = vals[all_metrics].copy()
for m in all_metrics:
    p = percentile_rank(vals[m])
    if m in INVERT_METRICS:
        p = 1 - p
    pct_df[m] = p

# -------------------------
# RENDER
# -------------------------
os.makedirs(OUT_DIR, exist_ok=True)

for idx, row in wide.iterrows():
    team = str(row["squadName"])
    team_dir = os.path.join(OUT_DIR, safe_filename(team))
    os.makedirs(team_dir, exist_ok=True)

    matches = row.get("matches", np.nan)

    for category, metrics in CATEGORY_METRICS.items():
        if len(metrics) < 3:
            continue

        scales = build_scales(vals, metrics)

        team_actual = pd.to_numeric(vals.loc[idx, metrics], errors="coerce").values.astype(float)

        team_norm = np.array([normalize(v, *scales[m][:2], invert=(m in INVERT_METRICS))
                              for v, m in zip(team_actual, metrics)], dtype=float)
        league_norm = np.array([normalize(scales[m][2], *scales[m][:2], invert=(m in INVERT_METRICS))
                                for m in metrics], dtype=float)

        # table data
        pcts = pct_df.loc[idx, metrics].values
        table_df = pd.DataFrame({
            "Metric": [nice_metric_label(m) for m in metrics],
            "Value": [format_value(v) for v in team_actual],
            "Percentile": [int(round(float(p)*100)) if pd.notna(p) else "" for p in pcts],
        }).drop_duplicates(subset=["Metric"], keep="first").reset_index(drop=True)

        fig = plt.figure(figsize=(18, 9), facecolor="white")
        draw_header(fig, team, category, matches)

        ax_radar = fig.add_axes([0.05, 0.10, 0.55, 0.78], polar=True)
        setup_radar(ax_radar, len(metrics))

        draw_metric_titles(ax_radar, metrics)
        draw_ring_value_labels_all(ax_radar, metrics, scales)

        # league avg filled + dashed outline
        plot_polygon(ax_radar, league_norm, color=LEAGUE_COLOR, linestyle="--",
                     linewidth=2.4, fill=True, fill_alpha=0.12, z=2)

        # team filled + solid outline
        plot_polygon(ax_radar, team_norm, color=TEAM_COLOR, linestyle="-",
                     linewidth=3.2, fill=True, fill_alpha=0.22, z=3)

        # legend
        fig.text(0.10, 0.09, "Team", color=TEAM_COLOR, fontsize=12, fontweight="bold")
        fig.text(0.145, 0.09, "•", color=TEXT_MID, fontsize=12)
        fig.text(0.155, 0.09, "League Avg", color=LEAGUE_COLOR, fontsize=12, fontweight="bold")

        ax_table = fig.add_axes([0.65, 0.20, 0.30, 0.62])
        draw_table(ax_table, table_df)

        fig.text(0.10, 0.05, "hudlstatsbomb", fontsize=20, fontweight="bold",
                 color="#F05A28", alpha=0.30)

        out_path = os.path.join(team_dir, f"{safe_filename(category)}.png")
        fig.savefig(out_path, dpi=220, bbox_inches="tight")
        plt.close(fig)

print(f"✅ Done. Saved PNGs to: {OUT_DIR}")


Final metric counts per category (after dedupe):
  - Attack: 10 -> ['xG', 'Goals', 'Shots', 'Shots Off Target', 'Assists', 'Opp. Bypassed', 'Opp. Bypassed Receiving', 'Succ. Passes', 'Unsuccessful Passes', 'Neutral Passes']
  - Defence: 10 -> ['Ball Wins', 'Ball Wins Interception', 'Defensive Touches', 'Won Ground Duels', 'Won Aerial Duels', 'Lost Ground Duels', 'Lost Aerial Duels', 'Number Of Presses', 'Number Of Presses Between The Lines', 'Number Of Presses During Opponent Build-Up']
  - Set Piece: 10 -> ['xG', 'Goals', 'Shots', 'Shots Off Target', 'Assists', 'Succ. Passes', 'Unsuccessful Passes', 'Neutral Passes', 'Opp. Bypassed', 'Opp. Bypassed Receiving']
  - Transition Attack: 10 -> ['xG', 'Goals', 'Shots', 'Shots Off Target', 'Assists', 'Opp. Bypassed', 'Opp. Bypassed Receiving', 'Succ. Passes', 'Unsuccessful Passes', 'Neutral Passes']
  - Transition Defence: 10 -> ['Ball Wins', 'Defensive Touches', 'Won Ground Duels', 'Won Aerial Duels', 'Lost Ground Duels', 'Lost Aerial Duels

In [18]:
"""
Squad Team Radars - Team vs League Average
Generates radars for each team compared to league average
Saves in team-specific folders
"""

import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# -------------------------
# CONFIGURATION
# -------------------------
INPUT_XLSX = "/Users/user/IMPECT/squad_kpis_iteration_1421.xlsx"
WIDE_SHEET = "squad_kpis_wide"
OUT_DIR = "/Users/user/IMPECT/outputs"
ITERATION_ID = 1421

# Color scheme - Team vs League
COLOR_TEAM = "#C8102E"  # Red for team
COLOR_LEAGUE = "#808080"  # Grey for league average
COLOR_BG = "#F5F5F0"  # Light beige/grey background
COLOR_GREY_LIGHT = "#E8E8E8"
COLOR_GREY_MID = "#BABABA"
COLOR_GREY_DARK = "#4A4A4A"
COLOR_TEXT = "#2C2C2C"
COLOR_HEADER_LINE = "#C8102E"  # Red header line

# Radar settings
USE_PER_MATCH = False  # Data is already per-match normalized
NUM_METRICS = 12
SCALE_QUANTILES = (0.05, 0.95)

# Metrics to invert (lower is better)
INVERT_METRICS = {
    "LOST_GROUND_DUELS_AT_PHASE_OUT_OF_POSSESSION",
    "LOST_AERIAL_DUELS_AT_PHASE_OUT_OF_POSSESSION",
    "LOST_GROUND_DUELS_AT_PHASE_DEFENSIVE_TRANSITION",
    "LOST_AERIAL_DUELS_AT_PHASE_DEFENSIVE_TRANSITION",
    "UNSUCCESSFUL_PASSES_AT_PHASE_SET_PIECE",
    "UNSUCCESSFUL_PASSES_AT_PHASE_ATTACKING_TRANSITION",
    "LOST_GROUND_DUELS_AT_PHASE_SET_PIECE",
    "LOST_AERIAL_DUELS_AT_PHASE_SET_PIECE",
}

# Phase tokens
PHASE_TOKEN = {
    "Attack": "AT_PHASE_IN_POSSESSION",
    "Defence": "AT_PHASE_OUT_OF_POSSESSION",
    "Set Piece": "AT_PHASE_SET_PIECE",
    "Attacking Transition": "AT_PHASE_ATTACKING_TRANSITION",
    "Defensive Transition": "AT_PHASE_DEFENSIVE_TRANSITION",
}

NON_METRIC_COLS = {"iterationId", "season", "squadId", "squadName", "matches"}

ALREADY_NORMALIZED_TOKENS = (
    "PERCENT", "PCT", "%", "RATIO", "RATE", "ACCURACY", "AVG", "MEAN", "MEDIAN", "PER_90", "PER90"
)

# -------------------------
# PREFERRED METRICS PER CATEGORY
# -------------------------
CATEGORY_PREFERRED = {
    "Attack": [
        "SHOT_XG_AT_PHASE_IN_POSSESSION",
        "GOALS_AT_PHASE_IN_POSSESSION",
        "SHOT_AT_GOAL_NUMBER_AT_PHASE_IN_POSSESSION",
        "ASSISTS_AT_PHASE_IN_POSSESSION",
        "BYPASSED_OPPONENTS_NUMBER_AT_PHASE_IN_POSSESSION",
        "BYPASSED_OPPONENTS_RECEIVING_NUMBER_AT_PHASE_IN_POSSESSION",
        "SUCCESSFUL_PASSES_AT_PHASE_IN_POSSESSION",
        "OFFENSIVE_TOUCHES_AT_PHASE_IN_POSSESSION",
        "BYPASSED_DEFENDERS_AT_PHASE_IN_POSSESSION",
        "REVERSE_PLAY_NUMBER_AT_PHASE_IN_POSSESSION",
        "UNSUCCESSFUL_PASSES_AT_PHASE_IN_POSSESSION",
        "SHOT_AT_GOAL_OFF_TARGET_NUMBER_AT_PHASE_IN_POSSESSION",
    ],
    "Defence": [
        "BALL_WIN_NUMBER_AT_PHASE_OUT_OF_POSSESSION",
        "BALL_WIN_NUMBER_BY_ACTION_INTERCEPTION",
        "DEFENSIVE_TOUCHES_AT_PHASE_OUT_OF_POSSESSION",
        "WON_GROUND_DUELS_AT_PHASE_OUT_OF_POSSESSION",
        "WON_AERIAL_DUELS_AT_PHASE_OUT_OF_POSSESSION",
        "Number of presses",
        "Number of presses between the lines",
        "Number of presses during opponent build-up",
        "Number of presses in counter press",
        "LOST_GROUND_DUELS_AT_PHASE_OUT_OF_POSSESSION",
        "LOST_AERIAL_DUELS_AT_PHASE_OUT_OF_POSSESSION",
    ],
    "Set Piece": [
        "SHOT_XG_AT_PHASE_SET_PIECE",
        "GOALS_AT_PHASE_SET_PIECE",
        "SHOT_AT_GOAL_NUMBER_AT_PHASE_SET_PIECE",
        "ASSISTS_AT_PHASE_SET_PIECE",
        "SUCCESSFUL_PASSES_AT_PHASE_SET_PIECE",
        "UNSUCCESSFUL_PASSES_AT_PHASE_SET_PIECE",
        "BYPASSED_OPPONENTS_NUMBER_AT_PHASE_SET_PIECE",
        "BYPASSED_OPPONENTS_RECEIVING_NUMBER_AT_PHASE_SET_PIECE",
        "BALL_WIN_NUMBER_AT_PHASE_SET_PIECE",
        "WON_AERIAL_DUELS_AT_PHASE_SET_PIECE",
        "LOST_AERIAL_DUELS_AT_PHASE_SET_PIECE",
        "DEFENSIVE_TOUCHES_AT_PHASE_SET_PIECE",
    ],
    "Attacking Transition": [
        "SHOT_XG_AT_PHASE_ATTACKING_TRANSITION",
        "GOALS_AT_PHASE_ATTACKING_TRANSITION",
        "SHOT_AT_GOAL_NUMBER_AT_PHASE_ATTACKING_TRANSITION",
        "ASSISTS_AT_PHASE_ATTACKING_TRANSITION",
        "BYPASSED_OPPONENTS_NUMBER_AT_PHASE_ATTACKING_TRANSITION",
        "BYPASSED_OPPONENTS_RECEIVING_NUMBER_AT_PHASE_ATTACKING_TRANSITION",
        "SUCCESSFUL_PASSES_AT_PHASE_ATTACKING_TRANSITION",
        "UNSUCCESSFUL_PASSES_AT_PHASE_ATTACKING_TRANSITION",
        "OFFENSIVE_TOUCHES_AT_PHASE_ATTACKING_TRANSITION",
        "BYPASSED_DEFENDERS_AT_PHASE_ATTACKING_TRANSITION",
        "REVERSE_PLAY_NUMBER_AT_PHASE_ATTACKING_TRANSITION",
        "SHOT_AT_GOAL_OFF_TARGET_NUMBER_AT_PHASE_ATTACKING_TRANSITION",
    ],
    "Defensive Transition": [
        "BALL_WIN_NUMBER_AT_PHASE_DEFENSIVE_TRANSITION",
        "DEFENSIVE_TOUCHES_AT_PHASE_DEFENSIVE_TRANSITION",
        "WON_GROUND_DUELS_AT_PHASE_DEFENSIVE_TRANSITION",
        "WON_AERIAL_DUELS_AT_PHASE_DEFENSIVE_TRANSITION",
        "LOST_GROUND_DUELS_AT_PHASE_DEFENSIVE_TRANSITION",
        "LOST_AERIAL_DUELS_AT_PHASE_DEFENSIVE_TRANSITION",
        "BALL_WIN_REMOVED_OPPONENTS_AT_PHASE_DEFENSIVE_TRANSITION",
        "BALL_WIN_ADDED_TEAMMATES_AT_PHASE_DEFENSIVE_TRANSITION",
    ],
}

# -------------------------
# HELPER FUNCTIONS
# -------------------------
def safe_filename(s: str) -> str:
    """Create safe filename from string"""
    s = re.sub(r"[^A-Za-z0-9 _-]+", "", str(s)).strip()
    return re.sub(r"\s+", "_", s)[:80]

def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Remove unnamed columns"""
    return df.loc[:, ~df.columns.astype(str).str.match(r"^Unnamed:")]

def is_already_normalized(colname: str) -> bool:
    """Check if metric is already normalized"""
    u = str(colname).upper()
    return any(tok in u for tok in ALREADY_NORMALIZED_TOKENS)

def nice_metric_label(col: str) -> str:
    """Clean metric labels for display"""
    s = str(col)
    
    # Remove phase tokens
    for tok in PHASE_TOKEN.values():
        s = s.replace(tok, "")
    
    s = s.replace("_AT_PHASE_", " ")
    s = s.replace("_BY_ACTION_", " ")
    s = s.replace("_NUMBER", "")
    s = s.replace("_", " ")
    s = re.sub(r"\s+", " ", s).strip()
    
    # Common abbreviations
    s = re.sub(r"\bSHOT XG\b", "xG", s, flags=re.IGNORECASE)
    s = re.sub(r"\bSHOT AT GOAL OFF TARGET\b", "Shots Off Target", s, flags=re.IGNORECASE)
    s = re.sub(r"\bSHOT AT GOAL\b", "Shots", s, flags=re.IGNORECASE)
    s = re.sub(r"\bSUCCESSFUL PASSES\b", "Succ. Passes", s, flags=re.IGNORECASE)
    s = re.sub(r"\bUNSUCCESSFUL PASSES\b", "Unsucc. Passes", s, flags=re.IGNORECASE)
    s = re.sub(r"\bBYPASSED OPPONENTS RECEIVING\b", "Opp. Bypassed Rec.", s, flags=re.IGNORECASE)
    s = re.sub(r"\bBYPASSED OPPONENTS\b", "Opp. Bypassed", s, flags=re.IGNORECASE)
    s = re.sub(r"\bBALL WIN REMOVED OPPONENTS\b", "Ball Win Rmv Opp.", s, flags=re.IGNORECASE)
    s = re.sub(r"\bBALL WIN ADDED TEAMMATES\b", "Ball Win Add Team.", s, flags=re.IGNORECASE)
    s = re.sub(r"\bBALL WIN\b", "Ball Wins", s, flags=re.IGNORECASE)
    
    # Title case
    s = s.title()
    s = s.replace("Xg", "xG").replace("Xa", "xA")
    s = s.replace("Rmv", "Rmv.").replace("Add", "Add.")
    return s

def format_value(v: float) -> str:
    """Format numeric values for display"""
    if pd.isna(v):
        return ""
    v = float(v)
    av = abs(v)
    if av >= 100:
        return f"{v:.0f}"
    if av >= 10:
        return f"{v:.1f}"
    if av >= 1:
        return f"{v:.2f}"
    return f"{v:.3f}"

def compute_per_match(df: pd.DataFrame, metric_cols, matches_col="matches") -> pd.DataFrame:
    """Convert raw counts to per-match values"""
    df2 = df.copy()
    m = pd.to_numeric(df2[matches_col], errors="coerce").replace(0, np.nan)
    for c in metric_cols:
        if is_already_normalized(c):
            continue
        df2[c] = pd.to_numeric(df2[c], errors="coerce") / m
    return df2

def percentile_rank(series: pd.Series) -> pd.Series:
    """Calculate percentile ranks"""
    s = pd.to_numeric(series, errors="coerce")
    return s.rank(pct=True, method="average")

def axis_bounds(series: pd.Series):
    """Calculate min, max, mean for scaling"""
    s = pd.to_numeric(series, errors="coerce").dropna()
    if s.empty:
        return (np.nan, np.nan, np.nan)
    mean = float(s.mean())
    if SCALE_QUANTILES is None:
        mn = float(s.min())
        mx = float(s.max())
    else:
        qlo, qhi = SCALE_QUANTILES
        mn = float(s.quantile(qlo))
        mx = float(s.quantile(qhi))
        if mx == mn:
            mn = float(s.min())
            mx = float(s.max())
    return (mn, mx, mean)

def normalize(v, mn, mx, invert=False):
    """Normalize value to 0-1 scale"""
    if pd.isna(v) or pd.isna(mn) or pd.isna(mx) or mx == mn:
        return np.nan
    x = (float(v) - float(mn)) / (float(mx) - float(mn))
    x = np.clip(x, 0, 1)
    if invert:
        x = 1 - x
    return x

def pick_metrics_for_category(all_cols, category, limit):
    """Select metrics for a category"""
    preferred = CATEGORY_PREFERRED.get(category, [])
    picked = []
    used_labels = set()
    
    for c in preferred:
        if c in all_cols:
            lab = nice_metric_label(c)
            if lab not in used_labels:
                picked.append(c)
                used_labels.add(lab)
        if len(picked) >= limit:
            return picked
    
    token = PHASE_TOKEN.get(category)
    if token:
        leftovers = [c for c in all_cols if isinstance(c, str) and token in c and c not in picked]
        leftovers = sorted(leftovers, key=lambda x: (0 if "_NUMBER_" in x else 1, len(x)))
        for c in leftovers:
            lab = nice_metric_label(c)
            if lab in used_labels:
                continue
            picked.append(c)
            used_labels.add(lab)
            if len(picked) >= limit:
                break
    
    return picked

# -------------------------
# DRAWING FUNCTIONS
# -------------------------
def setup_polar_ax(ax, n_spokes):
    """Setup polar axis for radar - clean style"""
    angles = np.linspace(0, 2*np.pi, n_spokes, endpoint=False)
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)
    ax.set_xticks(angles)
    ax.set_xticklabels([""] * n_spokes)
    ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
    ax.set_yticklabels([""] * 5)
    ax.set_ylim(0, 1)
    
    # Very light grey grid
    ax.grid(True, color='#D0D0D0', linewidth=0.5, linestyle='-', alpha=0.5)
    ax.spines["polar"].set_visible(False)
    ax.set_facecolor("white")

def plot_radar_data(ax, values, color, linewidth=2.5, alpha=0.3, label=None, linestyle='-'):
    """Plot radar data polygon - NO FILL for cleaner look"""
    vals = np.asarray(values, dtype=float)
    n = len(vals)
    ang = np.linspace(0, 2*np.pi, n, endpoint=False)
    ang_c = np.concatenate([ang, ang[:1]])
    vals_c = np.concatenate([vals, vals[:1]])
    
    # Only plot line, no fill
    ax.plot(ang_c, vals_c, color=color, linewidth=linewidth, label=label, zorder=10, linestyle=linestyle)

def add_spoke_labels(ax, metrics, scales, team_vals, league_vals):
    """Add metric labels and values on spokes"""
    n = len(metrics)
    angles = np.linspace(0, 2*np.pi, n, endpoint=False)
    
    for i, (ang, m) in enumerate(zip(angles, metrics)):
        # Metric label at the outer edge - tighter spacing
        label = nice_metric_label(m)
        x = np.cos(ang)
        ha = "center" if abs(x) < 0.15 else ("left" if x > 0 else "right")
        
        ax.text(
            ang, 1.15, label,
            fontsize=9, color=COLOR_TEXT, ha=ha, va="center",
            fontweight="normal"
        )
        
        # Add ring value labels along the spokes - fewer labels for cleaner look
        mn, mx, _ = scales[m]
        inv = (m in INVERT_METRICS)
        
        if not pd.isna(mn) and not pd.isna(mx) and mx != mn:
            # Show scale values at fewer radii for cleaner design
            for radius in [0.4, 0.8]:
                if inv:
                    val_at_radius = mx - radius * (mx - mn)
                else:
                    val_at_radius = mn + radius * (mx - mn)
                
                val_str = format_value(val_at_radius)
                
                # Alternate sides for readability
                if i % 2 == 0:
                    ax.text(
                        ang, radius + 0.02, val_str,
                        fontsize=6, color=COLOR_GREY_MID, 
                        ha="center", va="bottom",
                        fontweight="normal"
                    )

def create_comparison_table(ax, metrics, team_name, team_vals, team_pcts, league_vals):
    """Create comparison table - cleaner style matching reference"""
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis("off")
    
    # Light grey background
    ax.add_patch(Rectangle((0, 0), 1, 1, facecolor=COLOR_GREY_LIGHT, edgecolor="none"))
    
    # Header - minimal style
    header_h = 0.06
    ax.add_patch(Rectangle((0, 1-header_h), 1, header_h, 
                           facecolor=COLOR_GREY_LIGHT, edgecolor="none"))
    
    # Column headers - simple and clean
    ax.text(0.30, 1-header_h/2, "Metric", fontsize=10, fontweight="bold",
            color=COLOR_TEXT, ha="center", va="center")
    ax.text(0.65, 1-header_h/2, "Value", fontsize=10, fontweight="bold",
            color=COLOR_TEXT, ha="center", va="center")
    ax.text(0.85, 1-header_h/2, "Pct", fontsize=10, fontweight="bold",
            color=COLOR_TEXT, ha="center", va="center")
    
    # Rows - no alternating colors, just clean white rows with thin borders
    n = len(metrics)
    top = 1 - header_h
    row_h = (top - 0.02) / max(n, 1)
    y = top
    
    for i in range(n):
        y0 = y - row_h
        
        # White background for all rows
        ax.add_patch(Rectangle((0, y0), 1, row_h, 
                               facecolor="white", edgecolor=COLOR_GREY_MID, linewidth=0.3))
        
        # Metric name - left aligned
        metric_label = nice_metric_label(metrics[i])
        ax.text(0.05, y0 + row_h/2, metric_label, fontsize=8.5,
                color=COLOR_TEXT, ha="left", va="center")
        
        # Team value - centered
        ax.text(0.65, y0 + row_h/2, format_value(team_vals[i]), fontsize=8.5,
                color=COLOR_TEXT, ha="center", va="center")
        
        # Team percentile - centered, colored based on value
        pct = team_pcts[i]
        pct_text = "" if pd.isna(pct) else str(int(pct))
        pct_color = get_percentile_color(pct)
        ax.text(0.85, y0 + row_h/2, pct_text, fontsize=8.5,
                color=pct_color, ha="center", va="center", fontweight="bold")
        
        y = y0
    
    # Footer note
    ax.text(0.50, 0.01, "Radar: solid = team, dashed+filled = league avg",
            fontsize=8, color=COLOR_GREY_DARK, ha="center", va="bottom", style="italic")

def get_percentile_color(pct):
    """Get color based on percentile - blue for all percentiles matching reference"""
    if pd.isna(pct):
        return COLOR_TEXT
    # All percentiles in blue like the reference
    return "#4A90E2"  # Blue

# -------------------------
# MAIN EXECUTION
# -------------------------
def main():
    # Load data
    print("Loading data...")
    wide = pd.read_excel(INPUT_XLSX, sheet_name=WIDE_SHEET)
    wide = clean_columns(wide)
    
    # Validate required columns
    for col in ["squadName", "matches"]:
        if col not in wide.columns:
            raise ValueError(f"Missing required column '{col}'")
    
    wide["matches"] = pd.to_numeric(wide["matches"], errors="coerce")
    
    all_cols = list(wide.columns)
    
    # Select metrics per category - ALL 5 CATEGORIES
    CATEGORY_METRICS = {
        cat: pick_metrics_for_category(all_cols, cat, NUM_METRICS) 
        for cat in ["Attack", "Defence", "Set Piece", "Attacking Transition", "Defensive Transition"]
    }
    
    print("Metrics per category:")
    for cat, mets in CATEGORY_METRICS.items():
        print(f"  {cat}: {len(mets)} metrics")
    
    # Build union of all metrics
    all_metrics = sorted(set(m for mets in CATEGORY_METRICS.values() for m in mets))
    
    # Use values as-is (already per-match in the dataset)
    vals = wide.copy()
    # Note: Data is already per-match normalized, no further processing needed
    
    # Compute percentiles
    pct_df = vals[all_metrics].copy()
    for m in all_metrics:
        p = percentile_rank(vals[m])
        if m in INVERT_METRICS:
            p = 1 - p
        pct_df[m] = p * 100  # Store as 0-100
    
    # Calculate league averages
    league_avg = vals[all_metrics].mean()
    
    # Create output directory
    os.makedirs(OUT_DIR, exist_ok=True)
    
    # Generate radars for each team
    team_names = wide["squadName"].tolist()
    print(f"\nGenerating radars for {len(team_names)} teams...")
    
    for idx, team_name in enumerate(team_names):
        team_name_str = str(team_name)
        safe_team_name = safe_filename(team_name_str)
        
        # Create team folder
        team_folder = os.path.join(OUT_DIR, safe_team_name)
        os.makedirs(team_folder, exist_ok=True)
        
        team_matches = wide.loc[idx, "matches"]
        
        print(f"Creating radars for: {team_name_str}")
        
        for category, metrics in CATEGORY_METRICS.items():
            if len(metrics) < 3:
                continue
            
            # Build scales
            scales = {m: axis_bounds(vals[m]) for m in metrics}
            
            # Get team values
            team_actual = pd.to_numeric(vals.loc[idx, metrics], errors="coerce").values.astype(float)
            league_actual = league_avg[metrics].values.astype(float)
            
            # Normalize
            team_norm = np.array([
                normalize(v, *scales[m][:2], invert=(m in INVERT_METRICS))
                for v, m in zip(team_actual, metrics)
            ], dtype=float)
            
            league_norm = np.array([
                normalize(v, *scales[m][:2], invert=(m in INVERT_METRICS))
                for v, m in zip(league_actual, metrics)
            ], dtype=float)
            
            # Get percentiles
            team_pcts = pct_df.loc[idx, metrics].values
            
            # Create figure - cleaner design with beige background
            fig = plt.figure(figsize=(16, 9), facecolor=COLOR_BG)
            
            # Add horizontal red line at top
            fig.patches.append(plt.Rectangle((0, 0.93), 1, 0.004, 
                                            transform=fig.transFigure,
                                            facecolor=COLOR_HEADER_LINE, 
                                            edgecolor='none', 
                                            zorder=100))
            
            # Team name on left
            fig.text(0.03, 0.97, f"{team_name_str}", 
                    fontsize=22, color=COLOR_TEAM, fontweight="bold", va="top")
            fig.text(0.03, 0.945, f"Iteration {ITERATION_ID}",
                    fontsize=10, color=COLOR_GREY_DARK, va="top")
            
            # First metric name centered at top (as visual anchor)
            if len(metrics) > 0:
                first_metric = nice_metric_label(metrics[0])
                fig.text(0.50, 0.96, first_metric,
                        fontsize=11, color=COLOR_GREY_DARK, va="top", ha="center")
            
            # Radar category name on right
            fig.text(0.97, 0.97, f"{category} Radar",
                    fontsize=18, color=COLOR_TEXT, va="top", ha="right", fontweight="bold")
            fig.text(0.97, 0.945, f"{team_matches:.1f} matches",
                    fontsize=10, color=COLOR_GREY_DARK, va="top", ha="right")
            
            # Radar plot - larger
            ax_radar = fig.add_axes([0.03, 0.10, 0.52, 0.78], polar=True)
            setup_polar_ax(ax_radar, len(metrics))
            
            # Plot league average first (background)
            plot_radar_data(ax_radar, league_norm, COLOR_LEAGUE, 
                          linewidth=2.0, alpha=0.15, label="League Average",
                          linestyle='--')
            
            # Plot team (foreground)
            plot_radar_data(ax_radar, team_norm, COLOR_TEAM, 
                          linewidth=2.5, alpha=0.3, label=team_name_str)
            
            # Add spoke labels with values
            add_spoke_labels(ax_radar, metrics, scales, team_actual, league_actual)
            
            # Legend - bottom left style matching reference
            # No matplotlib legend, custom text instead
            fig.text(0.03, 0.07, "Team", fontsize=11, 
                    color=COLOR_TEAM, fontweight="bold", va="center")
            fig.text(0.08, 0.07, "•", fontsize=11, 
                    color=COLOR_GREY_DARK, va="center")
            fig.text(0.09, 0.07, "League Avg", fontsize=11,
                    color=COLOR_GREY_DARK, va="center")
            
            # Comparison table - adjusted position
            ax_table = fig.add_axes([0.60, 0.15, 0.37, 0.75])
            create_comparison_table(
                ax_table, metrics, team_name_str,
                team_actual, team_pcts, league_actual
            )
            
            # Footer - hudlstatsbomb branding
            fig.text(0.03, 0.02, "hudlstatsbomb", fontsize=18, 
                    color="#F39C12", fontweight="bold", alpha=0.4)
            
            # Save
            safe_category = safe_filename(category)
            filename = f"{safe_category}.png"
            out_path = os.path.join(team_folder, filename)
            
            fig.savefig(out_path, dpi=200, bbox_inches="tight", facecolor="white")
            plt.close(fig)
            print(f"  Saved: {safe_team_name}/{filename}")
    
    print(f"\n✅ Done! Radars saved to: {OUT_DIR}")
    print(f"   Created {len(team_names)} team folders")
    print(f"   Each team has 5 radars: Attack, Defence, Set Piece, Attacking Transition, Defensive Transition")

if __name__ == "__main__":
    main()

Loading data...
Metrics per category:
  Attack: 12 metrics
  Defence: 12 metrics
  Set Piece: 12 metrics
  Attacking Transition: 12 metrics
  Defensive Transition: 10 metrics

Generating radars for 16 teams...
Creating radars for: FC Slovan Liberec
  Saved: FC_Slovan_Liberec/Attack.png
  Saved: FC_Slovan_Liberec/Defence.png
  Saved: FC_Slovan_Liberec/Set_Piece.png
  Saved: FC_Slovan_Liberec/Attacking_Transition.png
  Saved: FC_Slovan_Liberec/Defensive_Transition.png
Creating radars for: AC Sparta Prag
  Saved: AC_Sparta_Prag/Attack.png
  Saved: AC_Sparta_Prag/Defence.png
  Saved: AC_Sparta_Prag/Set_Piece.png
  Saved: AC_Sparta_Prag/Attacking_Transition.png
  Saved: AC_Sparta_Prag/Defensive_Transition.png
Creating radars for: FC Viktoria Pilsen
  Saved: FC_Viktoria_Pilsen/Attack.png
  Saved: FC_Viktoria_Pilsen/Defence.png
  Saved: FC_Viktoria_Pilsen/Set_Piece.png
  Saved: FC_Viktoria_Pilsen/Attacking_Transition.png
  Saved: FC_Viktoria_Pilsen/Defensive_Transition.png
Creating radars for